Say we have an edge matrix $G$:

In [1]:
from projections import *
from graphReader import *
from helpers import *

G

array([[0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1],
       [1, 1, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]])

Of course, we can convert that to an "error matrix" of all elements that are NOT connected, like so:

In [2]:
M

array([[0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.],
       [1., 0., 1., 1., 0.]])

Now, we are looking for a clique. That is hard. However, we can start by looking for a quasi-clique, or a set of orthonormal vectors $X$ that satisfies $X^T M X = 0$.

How can we do this?

Well, assume we already have a set of vectors $X$ which we are trying to expand. Lets have a couple examples:

In [3]:
X  = np.array([[3.0/5, 4.0/5, 0, 0, 0], [(4.0/13), (-3.0/13), 12.0/13, 0, 0]]).T
X2 = np.array([[0, 0, 8.0/17, 15.0/17, 0], [0, 0, -15.0/17, 8.0/17, 0]]).T
X3 = np.array([[0, 0, 0, 0, 1.0]]).T

print("\n\nXs")
print(X)
print(X2)
print(X3)

print("\n\nOrthonormality Check - should be Id")
print(np.matmul(X.T, X).round(3))
print(np.matmul(X2.T, X2).round(3))
print(np.matmul(X3.T, X3).round(3))

def convolve(basis, Matrix):
    return np.matmul(basis.T, np.matmul(Matrix, basis))

print("\n\ncondition Check - should be 0")
print(convolve(X, M))
print(convolve(X2, M))
print(convolve(X3, M))



Xs
[[ 0.6         0.30769231]
 [ 0.8        -0.23076923]
 [ 0.          0.92307692]
 [ 0.          0.        ]
 [ 0.          0.        ]]
[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.47058824 -0.88235294]
 [ 0.88235294  0.47058824]
 [ 0.          0.        ]]
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]]


Orthonormality Check - should be Id
[[ 1. -0.]
 [-0.  1.]]
[[1. 0.]
 [0. 1.]]
[[1.]]


condition Check - should be 0
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
[[0.]]


Well, we are now looking for a vector $x$ which is orthogonal to $X$, to $MX$ and which satisfies $x^T M x = 0$.

We can work through these requirements one at at time. The projection onto the space spanned by $X$ is easy to write - it's just $XX^T$ - so the projection onto the kernel of that span is just $Id - XX^T$.

In [4]:
kerX  = np.eye(5) - np.matmul(X, X.T)
kerX2 = np.eye(5) - np.matmul(X2, X2.T)
kerX3 = np.eye(5) - np.matmul(X3, X3.T)
print(kerX.round(3))
print(kerX2.round(3))
print(kerX3.round(3))

[[ 0.545 -0.409 -0.284  0.     0.   ]
 [-0.409  0.307  0.213  0.     0.   ]
 [-0.284  0.213  0.148  0.     0.   ]
 [ 0.     0.     0.     1.     0.   ]
 [ 0.     0.     0.     0.     1.   ]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]


Next, we have to deal with $MX$, which is a little harder since it's not *ab initio* orthonormal.

Let's first look at $S = MX$.

In [5]:
S = np.matmul(M, X)
S2 = np.matmul(M, X2)
S3 = np.matmul(M, X3)
print(S.round(3))
print(S2.round(3))
print(S3.round(3))

[[0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]
 [1.4   0.077]
 [0.6   1.231]]
[[ 0.882  0.471]
 [ 0.882  0.471]
 [ 0.     0.   ]
 [ 0.     0.   ]
 [ 1.353 -0.412]]
[[1.]
 [0.]
 [1.]
 [1.]
 [0.]]


Well, clearly $S$ has two linearly independent components, but it is not orthonormal:

In [6]:
print(np.matmul(S.T, S))
print(np.matmul(S2.T, S2))
print(np.matmul(S3.T, S3))

[[2.32       0.84615385]
 [0.84615385 1.52071006]]
[[3.38754325 0.2733564 ]
 [0.2733564  0.61245675]]
[[3.]]


There is still, however, a simple enough procedure to obtain a projection onto the span of $S$. Start with the basic matrix $S S^T$, then use svd to compute $T v D = S S^T$. Set the nonzero elements of $v$ to 1, and compute!

In [7]:
P  = np.matmul(S, S.T)
P2 = np.matmul(S2, S2.T)
P3 = np.matmul(S3, S3.T)

T,  v,  D  = np.linalg.svd(P)
T2, v2, D2 = np.linalg.svd(P2)
T3, v3, D3 = np.linalg.svd(P3)
print(v)
print(v2.round(3))
print(v3.round(3))

[2.85613942 0.98457063 0.         0.         0.        ]
[3.414 0.586 0.    0.    0.   ]
[3. 0. 0. 0. 0.]


In [8]:
eps = 1e-7
w  = np.array([float(el > eps) for el in v])
w2 = np.array([float(el > eps) for el in v2])
w3 = np.array([float(el > eps) for el in v3])

# They're all the same
print(w)

Q  = np.matmul(T,  np.matmul(w*np.eye(5),  D))
Q2 = np.matmul(T2, np.matmul(w2*np.eye(5), D2))
Q3 = np.matmul(T3, np.matmul(w3*np.eye(5), D3))

print(Q.round(3))
print(Q2.round(3))
print(Q3.round(3))

[1. 1. 0. 0. 0.]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -0.]
 [ 0.  0.  0.  0.  1.]]
[[ 0.5  0.5  0.   0.   0. ]
 [ 0.5  0.5  0.   0.  -0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.  -0.   0.   0.   1. ]]
[[0.333 0.    0.333 0.333 0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.333 0.    0.333 0.333 0.   ]
 [0.333 0.    0.333 0.333 0.   ]
 [0.    0.    0.    0.    0.   ]]


Is $Q$ really a projection matrix? Well, it would appear so, since $Q^2 = Q$ in all cases:

In [9]:
print(np.matmul(Q,  Q).round(3))
print(np.matmul(Q2, Q2).round(3))
print(np.matmul(Q3, Q3).round(3))

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -0.]
 [ 0.  0.  0.  0.  1.]]
[[ 0.5  0.5  0.   0.   0. ]
 [ 0.5  0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [-0.  -0.   0.   0.   1. ]]
[[0.333 0.    0.333 0.333 0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.333 0.    0.333 0.333 0.   ]
 [0.333 0.    0.333 0.333 0.   ]
 [0.    0.    0.    0.    0.   ]]


So, the projection onto the kernel of the span of $MX$ is just $Id - Q$, so the projection onto the kernel of both $X$ and $MX$ is going to be
$$K = (Id - XX^T)(Id - Q)$$

In [10]:
kerMX = np.eye(5) - Q
kernel = np.matmul(kerX, kerMX)

kerMX2 = np.eye(5) - Q2
kernel2 = np.matmul(kerX2, kerMX2)

kerMX3 = np.eye(5) - Q3
kernel3 = np.matmul(kerX3, kerMX3)

print(kernel.round(3))
print(kernel2.round(3))
print(kernel3.round(3))

[[ 0.545 -0.409 -0.284  0.     0.   ]
 [-0.409  0.307  0.213  0.     0.   ]
 [-0.284  0.213  0.148  0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    -0.     0.   ]]
[[ 0.5 -0.5  0.   0.  -0. ]
 [-0.5  0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]]
[[ 0.667  0.    -0.333 -0.333  0.   ]
 [ 0.     1.     0.     0.     0.   ]
 [-0.333  0.     0.667 -0.333  0.   ]
 [-0.333  0.    -0.333  0.667  0.   ]
 [ 0.     0.     0.     0.     0.   ]]


Well, now all that is left is to find the space that $K$ spans, and look at the action of $M$ on this space only:

In [35]:
Bt,  u,  B  = np.linalg.svd(kernel)
B2t, u2, B2 = np.linalg.svd(kernel2)
B3t, u3, B3 = np.linalg.svd(kernel3)

# THey're all the same
print(u.round(3))
print(B)

ind  = [i for i in range(5) if u[i] > eps]
ind2 = [i for i in range(5) if u2[i] > eps]
ind3 = [i for i in range(5) if u3[i] > eps]

# This is the parametrization of the space we care about:
b = B[ind, :].T
b2 = B2[ind2, :].T
b3 = B3[ind3, :].T

print("\n\nBasis vector(s) of each new space")
print(b.round(3))
print(b2.round(3))
print(b3.round(3))

# This is the action of M on only the space we care about:
print("\n\nAction of M on each subspace:")
M1 = np.matmul(b.T,  np.matmul(M, b))
M2 = np.matmul(b2.T, np.matmul(M, b2))
M3 = np.matmul(b3.T, np.matmul(M, b3))

print(M1.round(3))
print(M2.round(3))
print(M3.round(3))

[1. 0. 0. 0. 0.]
[[-0.73846154  0.55384615  0.38461538  0.          0.        ]
 [-0.          0.          0.         -0.90955199 -0.41559015]
 [-0.05450082 -0.61755479  0.78463733  0.          0.        ]
 [ 0.          0.          0.         -0.41559015  0.90955199]
 [-0.67208944 -0.55846264 -0.48622553 -0.         -0.        ]]


Basis vector(s) of each new space
[[-0.738]
 [ 0.554]
 [ 0.385]
 [ 0.   ]
 [ 0.   ]]
[[-0.707]
 [ 0.707]
 [ 0.   ]
 [ 0.   ]
 [ 0.   ]]
[[ 0.     0.816  0.   ]
 [-1.     0.     0.   ]
 [-0.    -0.408 -0.707]
 [ 0.    -0.408  0.707]
 [ 0.     0.     0.   ]]


Action of M on each subspace:
[[0.]]
[[-0.]]
[[ 0.     0.408 -0.707]
 [ 0.408 -0.667  0.577]
 [-0.707  0.577  0.   ]]


Now that we have the action of $M$ restricted to the subspace in question, all that's left is to use the "nullproject" function from projections (not discussed here, but simple enough to derive) to find vectors in each subspace which satisfy $x^TMx = 0$:

In [37]:
v1 = np.random.random(M1.shape[0])*2 - 1
v2 = np.random.random(M2.shape[0])*2 - 1
v3 = np.random.random(M3.shape[0])*2 - 1

v1 = v1/norm(v1)
v2 = v2/norm(v2)
v3 = v3/norm(v3)

u1 = nullproject(M1, v1)
u2 = nullproject(M2, v2)
u3 = nullproject(M3, v3)

print(u1)
print(u2)
print(u3)

[1.]
[-1.]
[-0.26854502 -0.32889914  0.90537778]


And now we can express these vectors in the original basis:

In [39]:
x1 = np.matmul(b, u1).reshape(5, 1)
x2 = np.matmul(b2, u2).reshape(5, 1)
x3 = np.matmul(b3, u3).reshape(5, 1)

print("\n\nNew vectors x:")
print(x1.round(3))
print(x2.round(3))
print(x3.round(3))

print("\n\nTesting property:")
print(np.matmul(x3.T, np.matmul(M, x3)))



New vectors x:
[[-0.738]
 [ 0.554]
 [ 0.385]
 [ 0.   ]
 [ 0.   ]]
[[ 0.707]
 [-0.707]
 [ 0.   ]
 [ 0.   ]
 [-0.   ]]
[[-0.269]
 [ 0.269]
 [-0.506]
 [ 0.774]
 [ 0.   ]]


Testing property:
[[0.]]


All that's really left is to add this to the old list of vectors, and we're done!

In [42]:
Y  = np.concatenate((X, x1), axis=1)
Y2 = np.concatenate((X2, x2), axis=1)
Y3 = np.concatenate((X3, x3), axis=1)

print(Y.round(3))
print(Y2.round(3))
print(Y3.round(3))

[[ 0.6    0.308 -0.738]
 [ 0.8   -0.231  0.554]
 [ 0.     0.923  0.385]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]
[[ 0.     0.     0.707]
 [ 0.     0.    -0.707]
 [ 0.471 -0.882  0.   ]
 [ 0.882  0.471  0.   ]
 [ 0.     0.    -0.   ]]
[[ 0.    -0.269]
 [ 0.     0.269]
 [ 0.    -0.506]
 [ 0.     0.774]
 [ 1.     0.   ]]


What now? It's not clear. It's worth playing with this stuff, though:

In [94]:
print(np.matmul(Y, Y.T).round(3))
print(np.matmul(Y2, Y2.T).round(3))

D, k, C = svd(np.matmul(Y2, Y2.T))
print(C.round(3))
# It really looks like novel has more than one prominent hole in the rest of the structure:
novel = np.matmul(C, np.matmul(M, C.T))
print(novel.round(3))
connections = np.sign(abs(novel.round(6)))

print(np.sign(abs(novel.round(6))))
corr = np.eye(5)
corr[0, 1] = 1
corr[1, 0] = 1
corr[0, 0] = 0
corr[1, 1] = 0
tester = np.matmul(corr, np.matmul(M, corr))
# print(M)
# print(corr)
print(M)
print(connections)
print(np.sign(novel.round(6)))
print(M == np.sign(abs(novel.round(6))))

[[ 1.  0. -0.  0.  0.]
 [ 0.  1. -0.  0.  0.]
 [-0. -0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
[[ 0.5 -0.5  0.   0.  -0. ]
 [-0.5  0.5  0.   0.   0. ]
 [ 0.   0.   1.  -0.   0. ]
 [ 0.   0.  -0.   1.   0. ]
 [-0.   0.   0.   0.   0. ]]
[[-0.    -0.    -0.999  0.036  0.   ]
 [-0.707  0.707  0.     0.     0.   ]
 [ 0.     0.     0.036  0.999  0.   ]
 [ 0.707  0.707  0.     0.    -0.   ]
 [-0.    -0.     0.     0.    -1.   ]]
[[ 0.    -0.     0.     0.051  0.963]
 [-0.    -0.     0.     0.     0.707]
 [ 0.     0.     0.     1.413 -1.036]
 [ 0.051  0.     1.413 -0.    -0.707]
 [ 0.963  0.707 -1.036 -0.707  0.   ]]
[[0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]
[[0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 1. 1. 0.]]
[[0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]
[[ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1. -1.]
 [ 1.  0.  1.  0. -

With any luck, that sort of transformation will be sufficient. No idea how to guarantee the non-computed rows behave so nicely, though. We'll have to see until after I code up the "get a quasi-clique" formula. Maybe there's a good gradient-descent algorithm that takes one from a quasi-clique to a clique without violating any properties.

The golden goose would be getting "corr" from the matrix Y2. I wonder if that is possible.

Meanwhile, some exploration:

In [82]:
a, b, c = svd(corr)
#print(b)
#print(c)
#print(a)
print(np.matmul(D, C).round(3))
#print(corr)

[[ 0. -1.  0.  0.  0.]
 [-1. -0.  0.  0.  0.]
 [ 0.  0.  1. -0.  0.]
 [ 0.  0. -0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


In [74]:
print(sum(sum(M)))
print(sum(sum(connections)))

10.0
12.0
